## Проверка данных Цифромеда и прочих

In [7]:
import pandas as pd
import numpy as np
import os
import sys
from datetime import datetime
from datetime import timedelta

# игнорирование предупреждения о формате при импорте из excel
import warnings

warnings.simplefilter("ignore")
# пути и списки файлов
# файлы берём непосредственно из хранилища нового бота посещений
e_path = "C:/work/new_bot/new/e"
s_path = "C:/work/new_bot/new/s"
dm_path = "C:/work/new_bot/new/dm"
vis_reg_ref_path = "C:/work/new_bot/new/vis_reg_ref.csv"
e_file_list = os.listdir(e_path)
s_file_list = os.listdir(s_path)
dm_file_last = os.listdir(dm_path + "/" + max(os.listdir(dm_path)))
# выгрузки ПОС
pos_path = "C:/work/pos_bot/POS"
pos_dir_last = os.listdir(pos_path + "/" + max(os.listdir(pos_path)))
pos_reg_ref_path = "/".join(pos_path.split("/")[:-1]) + "/pos_reg_ref.csv"

# типы ошибок
t_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
    "Время уже занято другим пациентом",
    "Услуга посмертно не оказывается",
    "Время начала приема уже прошло",
]
tdb_error_list = [
    "Ошибка валидации данных пациента",
    "Ошибка «РМИС не ответил»",
    "Внутренняя ошибка РМИС при создании записи",
    "Сервис МИС приостановлен",
    "Ошибка валидации схемы",
]
b_list = [
    "Пациент уже записан на это время к другому специалисту",
    "Пациент уже записан к специалисту в этот день",
    "Выбранное время доступно только для записи пациентов определенного возраста",
]
o_error_list = [
    "Значение не найдено",
]
r_error_list = [
    "Ошибка «СМЭВ-timeout»",
    "Внутренняя ошибка концентратора при создании записи",
    "Ошибка во входных параметрах при записи на прием",
]
any_error_list = [
    "Ошибка доставки формы при записи на прием",
    "Истекло время сессии пользователя",
    "Ошибка соответствия ответа запросу МИС",
]
draft_list = ["Ошибка отсутсвует"]
# словарь списков типов ошибок
e_type_dict = {
    "t": t_error_list,
    "tdb": tdb_error_list,
    "b": b_list,
    "o": o_error_list,
    "r": r_error_list,
    "any": any_error_list,
    "draft": draft_list,
}


def error_table(df):
    """функция принимает только (!) df_error_yesterday или df_error_today из возвращает df ошибок"""
    df_tmp_o = df[df["final_state"].isin(o_error_list)]
    df_tmp_o["type"] = "орг. дашборд"
    df_tmp_t = df[df["final_state"].isin(t_error_list)]
    df_tmp_t["type"] = "технические всего"
    df_tmp_tdb = df[df["final_state"].isin(tdb_error_list)]
    df_tmp_tdb["type"] = "технические дашборд"
    df_tmp_b = df[df["final_state"].isin(b_list)]
    df_tmp_b["type"] = "бизнес-правила"
    df_tmp_r = df[df["final_state"].isin(r_error_list)]
    df_tmp_r["type"] = "фэр дашборд"
    df_tmp_any = df[df["final_state"].isin(any_error_list)]
    df_tmp_any["type"] = "прочая"
    df_tmp_draft = df[df["final_state"].isin(draft_list)]
    df_tmp_draft["type"] = "прервано дашборд"
    df_error_type = pd.concat(
        [df_tmp_o, df_tmp_t, df_tmp_tdb, df_tmp_b, df_tmp_r, df_tmp_any, df_tmp_draft]
    ).reset_index(drop=True)
    df_error_type = (
        df_error_type.pivot_table(
            index="Название субъекта РФ",
            columns="type",
            values="session_id",
            aggfunc="count",
        )
        .reset_index()
        .fillna(0)
        .rename_axis(None, axis=1)
    )
    df_error_type = df_error_type.astype(
        {
            "бизнес-правила": "int",
            "орг. дашборд": "int",
            "прервано дашборд": "int",
            "технические всего": "int",
            "технические дашборд": "int",
            "фэр дашборд": "int",
        }
    )
    return df_error_type

def badsim(df, sim):
    """проверка на наличие в текстовых полях недопустимого символа"""
    print(f"Записи, содержащие '{sim}':")
    for col in df.columns.tolist():
        if df.dtypes[col] == "object":
            try:
                bad_str = df[df[col].str.contains(sim)][col].unique()
            except Exception as e:
                pass
            if len(bad_str) != 0:
                print(f"\tПОЛЕ: {col}")
                print(bad_str)

### Ошибки за две последние даты

In [2]:
%%time
# ошибки записи (два последние файла)
m=0
for file_name in e_file_list[-2:]:
    if m == 0:
        file_name_for_error_count_yesterday = file_name
    else:
        file_name_for_error_count_today = file_name
    df_tmp = pd.read_csv(e_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк:\t\t\t{df_tmp.shape[0]}")
    cnt_c = "верно" if df_tmp.shape[1] == 13 else "ошибка"
    print(f"\t\tвсего столбцов:\t\t\t{df_tmp.shape[1]} ({cnt_c})")
    print("\tПропуски в столбцах, где их быть не должно:")
    df_nonan = df_tmp[
        [
            "session_id",
            "Название субъекта РФ",
            "step_id",
            "step_name",
            "error_code",
            "final_state",
            "create_ts",
            "Тип записи",
            "referral_flg",
        ]
    ]
    n = 0
    for column in df_nonan.columns:
        if df_nonan[column].isnull().sum() != 0:
            print(
                f"\t\tстолбец '{column}':\t\t {df_nonan[column].isnull().sum()} ({round(df_nonan[column].isnull().sum()*100/df_nonan.shape[0], 2)} %)"
            )
            n += 1
    if n == 0:
        print("\t\tотсутствуют")
    print(
        "\tОшибки данных (количество и доля от числа строк на соответствующем этапе):"
    )
    # число строк на соответствующих этапах
    len_ValidatePerson = df_tmp[df_tmp["step_name"] == "ValidatePerson"].shape[0]
    len_MO = df_tmp[df_tmp["step_name"] == "MO"].shape[0]
    len_ServiceOrSpecs = df_tmp[df_tmp["step_name"] == "ServiceOrSpecs"].shape[0]
    len_Slot = df_tmp[df_tmp["step_name"] == "Slot"].shape[0]
    len_Book = df_tmp[df_tmp["step_name"] == "Book"].shape[0]

    """
    На этапе валидации пациента
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ValidatePerson = df_tmp[
        (df_tmp["step_name"] == "ValidatePerson")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора МО
    есть данные oid МО
    ИЛИ есть данные о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_MO = df_tmp[
        (df_tmp["step_name"] == "MO")
        & (
            df_tmp["mo_oid"].notnull()
            | df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе выбора специальности врача
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ есть данные о специальности врача
    ИЛИ есть данные о слоте
    """
    df_ServiceOrSpecs = df_tmp[
        (df_tmp["step_name"] == "ServiceOrSpecs")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """"
    На этапе выбора слота
    нет данных об oid МО
    ИЛИ нет данных о названии МО
    ИЛИ нет данных о специальтности врача
    ИЛИ есть данные о времени слота
    """
    df_Slot = df_tmp[
        (df_tmp["step_name"] == "Slot")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | df_tmp["slot_ts"].notnull()
        )
    ]
    """
    На этапе записи
    нет любых данных (ИЛИ)
    """
    df_Book = df_tmp[
        (df_tmp["step_name"] == "Book")
        & (
            ~df_tmp["mo_oid"].notnull()
            | ~df_tmp["mo_name"].notnull()
            | ~df_tmp["Post_name"].notnull()
            | ~df_tmp["slot_ts"].notnull()
        )
    ]

    if df_ValidatePerson.shape[0] >= 0:
        print(
            f"\t\tValidatePerson_uncorrected:\t{df_ValidatePerson.shape[0]}\t({round(df_ValidatePerson.shape[0]*100/len_ValidatePerson, 2)} %)"
        )
    if df_MO.shape[0] >= 0:
        print(
            f"\t\tMO_uncorrected:\t\t\t{df_MO.shape[0]}\t({round(df_MO.shape[0]*100/len_MO, 2)} %)"
        )
    if df_ServiceOrSpecs.shape[0] >= 0:
        print(
            f"\t\tServiceOrSpecs_uncorrected:\t{df_ServiceOrSpecs.shape[0]}\t({round(df_ServiceOrSpecs.shape[0]*100/len_ServiceOrSpecs, 2)} %)"
        )
    if df_Slot.shape[0] >= 0:
        print(
            f"\t\tSlot_uncorrected:\t\t{df_Slot.shape[0]}\t({round(df_Slot.shape[0]*100/len_Slot, 2)} %)"
        )
    if df_Book.shape[0] >= 0:
        print(
            f"\t\tBook_uncorrected:\t\t{df_Book.shape[0]}\t({round(df_Book.shape[0]*100/len_Book, 2)} %)"
        )
    if (
        df_ValidatePerson.shape[0]
        + df_MO.shape[0]
        + df_ServiceOrSpecs.shape[0]
        + df_Slot.shape[0]
        + df_Book.shape[0]
        == 0
    ):
        print("\t\tотсутствуют")

    # субъекты, которых нет на дашборде
    df_tmp_3 = df_tmp[df_tmp['Тип записи'] == 3]

    # данные для сравнительного расчёта
    if m == 0:
        df_error_yesterday = df_tmp_3.copy()
    else:
        df_error_today = df_tmp_3.copy()
    
    df_nosub = df_tmp_3[
        (df_tmp["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp["Название субъекта РФ"] == "г. Байконур")
    ][["Название субъекта РФ", "final_state"]]
    df_nosub["value"] = 1
    # ошибки, которых нет на дашборде
    df_nosub_e = (
        df_nosub.groupby(["Название субъекта РФ", "final_state"])
        .sum()
        .reset_index()[["final_state", "value"]]
    )
    no_db_e_dict = {}
    for key, value in e_type_dict.items():
        no_db_e_dict[str(key + "_ndb")] = df_nosub_e[
            df_nosub_e["final_state"].isin(value)
        ]["value"].sum()

    df_errors = pd.DataFrame(
        df_tmp[df_tmp["Тип записи"] == 3].groupby("final_state")["session_id"].count()
    ).reset_index()
    print('\tКоличество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):')
    o_e = df_errors[df_errors["final_state"].isin(o_error_list)]["session_id"].sum()
    print(f'\t\tорганизационные ошибки:\t\t{o_e}\t({o_e - no_db_e_dict["o_ndb"]})')
    t_e = df_errors[df_errors["final_state"].isin(t_error_list)]["session_id"].sum()
    t_e_db = (
        df_errors[df_errors["final_state"].isin(tdb_error_list)]["session_id"].sum()
        - no_db_e_dict["tdb_ndb"] # было t_ndb
    )
    print(f"\t\tтехнические ошибки:\t\t{t_e}\t({t_e_db})")
    r_e = df_errors[df_errors["final_state"].isin(r_error_list)]["session_id"].sum()
    print(f'\t\tошибки ФЭР:\t\t\t{r_e}\t({r_e - no_db_e_dict["r_ndb"]})')
    any_e = df_errors[df_errors["final_state"].isin(any_error_list)]["session_id"].sum()
    print(f'\t\tпрочие ошибки:\t\t\t{any_e}\t({any_e - no_db_e_dict["any_ndb"]})')
    draft = df_errors[df_errors["final_state"].isin(draft_list)]["session_id"].sum()
    print(f'\t\tчерновики:\t\t\t{draft}\t({draft - no_db_e_dict["draft_ndb"]})')
    b = df_errors[df_errors["final_state"].isin(b_list)]["session_id"].sum()
    print(f'\t\tбизнес-правила:\t\t\t{b}\t({b - no_db_e_dict["b_ndb"]})')
    print("\tСубъекты, не отображаемые на дашборде:")
    if df_nosub.empty:
        print("\t\tотсутствуют")
    else:
        print(
            df_nosub.groupby(["Название субъекта РФ", "final_state"])
            .sum()
            .reset_index()
            .to_string(index=False)
        )
    m+=1
badsim(df_error_yesterday, "_")
badsim(df_error_today, "_")
print("-----------------------------------------------------------------------------------------------")
print(f"Вчерашний файл:\t\t{file_name_for_error_count_yesterday}")
print(f"Актуальный файл:\t{file_name_for_error_count_today}")
print("-----------------------------------------------------------------------------------------------")
    # break


Файл 12.02.2024.csv:
		всего строк:			720027
		всего столбцов:			13 (верно)
	Пропуски в столбцах, где их быть не должно:
		столбец 'Название субъекта РФ':		 6 (0.0 %)
	Ошибки данных (количество и доля от числа строк на соответствующем этапе):
		ValidatePerson_uncorrected:	5	(0.01 %)
		MO_uncorrected:			0	(0.0 %)
		ServiceOrSpecs_uncorrected:	5277	(3.64 %)
		Slot_uncorrected:		3250	(1.76 %)
		Book_uncorrected:		77	(0.4 %)
	Количество ошибок записи на приём (тип записи "К врачу"; рассылка (дашборд)):
		организационные ошибки:		71509	(71458)
		технические ошибки:		65879	(63514)
		ошибки ФЭР:			42817	(42797)
		прочие ошибки:			0	(0)
		черновики:			523673	(523471)
		бизнес-правила:			13260	(13250)
	Субъекты, не отображаемые на дашборде:
                      Название субъекта РФ                                                                 final_state  value
Федеральное медико-биологическое агентство                                  Внутренняя ошибка РМИС при создании записи      7
Федер

In [3]:
e_file_list[-2:]

['12.02.2024.csv', '13.02.2024.csv']

#### Ошибки по регионам на последнюю дату

In [4]:
print(f"Файл: {file_name_for_error_count_today}")
error_table(df_error_today)

Файл: 13.02.2024.csv


,Название субъекта РФ,бизнес-правила,орг. дашборд,прервано дашборд,технические всего,технические дашборд,фэр дашборд
0,Алтайский край,3,367,24952,1334,1334,808
1,Амурская область,83,9,2643,352,350,26
2,Архангельская область,48,455,11066,568,538,587
3,Астраханская область,9,79,279,2,2,4
4,Белгородская область,21,37,520,539,539,760
...,...,...,...,...,...,...,...
81,Ямало-Ненецкий автономный округ,15,111,299,1,1,2
82,Ярославская область,74,1817,3274,155,153,507
83,г. Москва,0,787,10387,5040,5040,118
84,г. Санкт-Петербург,216,1437,14204,678,653,1821


#### Сравнение ошибок "вчера-сегодня"

In [5]:
df_e_y = error_table(df_error_yesterday)
df_e_t = error_table(df_error_today)
df_e_yt = df_e_t.merge(df_e_y, on="Название субъекта РФ", how="outer").fillna(0)
df_e_yt = df_e_yt.rename(
    columns={
        "бизнес-правила_x": "бп отч",
        "орг. дашборд_x": "оДБ отч",
        "прервано дашборд_x": "чДБ отч",
        "технические всего_x": "т отч",
        "технические дашборд_x": "тДБ отч",
        "фэр дашборд_x": "фДБ отч",
        "бизнес-правила_y": "бп прш",
        "орг. дашборд_y": "оДБ прш",
        "прервано дашборд_y": "чДБ прш",
        "технические всего_y": "т прш",
        "технические дашборд_y": "тДБ прш",
        "фэр дашборд_y": "фДБ прш",
    }
)
df_e_yt = df_e_yt.astype(
    {
        "бп отч": "int",
        "оДБ отч": "int",
        "чДБ отч": "int",
        "т отч": "int",
        "тДБ отч": "int",
        "фДБ отч": "int",
        "бп прш": "int",
        "оДБ прш": "int",
        "чДБ прш": "int",
        "т прш": "int",
        "тДБ прш": "int",
        "фДБ прш": "int",
    }
)
df_e_yt = df_e_yt[
    [
        "Название субъекта РФ",
        "бп отч",
        "бп прш",
        "оДБ отч",
        "оДБ прш",
        "чДБ отч",
        "чДБ прш",
        "т отч",
        "т прш",
        "тДБ отч",
        "тДБ прш",
        "фДБ отч",
        "фДБ прш",
    ]
]
df_e_yt.insert(3, "Δбп", df_e_yt["бп отч"] - df_e_yt["бп прш"])
df_e_yt.insert(6, "ΔоДБ", df_e_yt["оДБ отч"] - df_e_yt["оДБ прш"])
df_e_yt.insert(9, "ΔчДБ", df_e_yt["чДБ отч"] - df_e_yt["чДБ прш"])
df_e_yt.insert(12, "Δт", df_e_yt["т отч"] - df_e_yt["т прш"])
df_e_yt.insert(15, "ΔтДБ", df_e_yt["тДБ отч"] - df_e_yt["тДБ прш"])
df_e_yt["ΔфДБ"] = df_e_yt["фДБ отч"] - df_e_yt["фДБ прш"]

df_e_yt

,Название субъекта РФ,бп отч,бп прш,Δбп,оДБ отч,оДБ прш,ΔоДБ,чДБ отч,чДБ прш,ΔчДБ,т отч,т прш,Δт,тДБ отч,тДБ прш,ΔтДБ,фДБ отч,фДБ прш,ΔфДБ
0,Алтайский край,3,5,-2,367,387,-20,24952,30276,-5324,1334,522,812,1334,522,812,808,1128,-320
1,Амурская область,83,126,-43,9,18,-9,2643,3691,-1048,352,463,-111,350,461,-111,26,56,-30
2,Архангельская область,48,63,-15,455,582,-127,11066,14521,-3455,568,160,408,538,119,419,587,857,-270
3,Астраханская область,9,9,0,79,85,-6,279,324,-45,2,5,-3,2,5,-3,4,26,-22
4,Белгородская область,21,16,5,37,52,-15,520,686,-166,539,468,71,539,468,71,760,1027,-267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Ямало-Ненецкий автономный округ,15,57,-42,111,381,-270,299,1870,-1571,1,17,-16,1,16,-15,2,452,-450
82,Ярославская область,74,78,-4,1817,2305,-488,3274,3721,-447,155,201,-46,153,199,-46,507,604,-97
83,г. Москва,0,0,0,787,1109,-322,10387,11163,-776,5040,5749,-709,5040,5749,-709,118,275,-157
84,г. Санкт-Петербург,216,285,-69,1437,2084,-647,14204,16269,-2065,678,2395,-1717,653,2369,-1716,1821,2540,-719


### Успехи за две последние даты

In [6]:
%%time
k=0
# успешные записи (два последние файла)
for file_name in s_file_list[-2:]:
    df_tmp_s = pd.read_csv(s_path + "/" + file_name, sep=";")
    print(f"\nФайл {file_name}:")
    print(f"\t\tвсего строк:\t\t{df_tmp_s.shape[0]}")
    cnt_s = "верно" if df_tmp_s.shape[1] == 9 else "ошибка"
    print(f"\t\tвсего столбцов:\t\t{df_tmp_s.shape[1]} ({cnt_s})")
    print("\tПропуски данных (всего (доля от общего числа строк)):")
    # пропуски данных
    missing_s_dict = {}
    for column in df_tmp_s.columns:
        missing_s_dict[column] = [
            df_tmp_s[column].isnull().sum(),
            round(df_tmp_s[column].isnull().sum() * 100 / df_tmp_s.shape[0], 2),
        ]
    print(f'\t\tДата:\t\t\t{missing_s_dict["Дата"][0]}\t({missing_s_dict["Дата"][1]} %)')
    print(
        f'\t\tsession_id:\t\t{missing_s_dict["session_id"][0]}\t({missing_s_dict["session_id"][1]} %)'
    )
    print(
        f'\t\tНазвание субъекта РФ:\t{missing_s_dict["Название субъекта РФ"][0]}\t({missing_s_dict["Название субъекта РФ"][1]} %)'
    )
    print(
        f'\t\tsp_oid:\t\t\t{missing_s_dict["sp_oid"][0]}\t({missing_s_dict["sp_oid"][1]} %)'
    )
    print(
        f'\t\tsp_name:\t\t{missing_s_dict["sp_name"][0]}\t({missing_s_dict["sp_name"][1]} %)'
    )
    print(
        f'\t\tpost_name:\t\t{missing_s_dict["post_name"][0]}\t({missing_s_dict["post_name"][1]} %)'
    )
    print(
        f'\t\tslot_ts:\t\t{missing_s_dict["slot_ts"][0]}\t({missing_s_dict["slot_ts"][1]} %)'
    )
    print(
        f'\t\tcreate_ts:\t\t{missing_s_dict["create_ts"][0]}\t({missing_s_dict["create_ts"][1]} %)'
    )
    print(
        f'\t\tТип записи:\t\t{missing_s_dict["Тип записи"][0]}\t({missing_s_dict["Тип записи"][1]} %)'
    )
    # субъекты, которых нет на дашборде
    df_nosub_s = df_tmp_s[
        (df_tmp_s["Название субъекта РФ"] == "Федеральное медико-биологическое агентство")
        | (df_tmp_s["Название субъекта РФ"] == "г. Байконур")
    ]
    print('\tКоличество успешных записей (тип записи "К врачу"; рассылка (дашборд)):')
    print(
        f'\t\t{df_tmp_s[df_tmp_s["Тип записи"] == 3].shape[0]} ({df_tmp_s[df_tmp_s["Тип записи"] == 3].shape[0] - df_nosub_s[df_nosub_s["Тип записи"] == 3].shape[0]})'
    )
    print("\tСубъекты, не отображаемые на дашборде:")
    if df_nosub_s.empty:
        print("\t\tотсутствуют")
    else:
        print(
            df_nosub_s.groupby(["Название субъекта РФ"])["session_id"]
            .count()
            .reset_index()
            .to_string(index=False)
        )
    df_tmp_s_3 = df_tmp_s[df_tmp_s["Тип записи"] == 3]
    if k == 0:
        df_s_yesterday = df_tmp_s_3.copy()
    else:
        df_s_today = df_tmp_s_3.copy()
    k+=1
    # break
badsim(df_s_yesterday, "_")
badsim(df_s_today, "_")


Файл И38_успешные_сессии_20240212.csv:
		всего строк:		172964
		всего столбцов:		9 (верно)
	Пропуски данных (всего (доля от общего числа строк)):
		Дата:			0	(0.0 %)
		session_id:		0	(0.0 %)
		Название субъекта РФ:	0	(0.0 %)
		sp_oid:			2557	(1.48 %)
		sp_name:		0	(0.0 %)
		post_name:		0	(0.0 %)
		slot_ts:		0	(0.0 %)
		create_ts:		0	(0.0 %)
		Тип записи:		0	(0.0 %)
	Количество успешных записей (тип записи "К врачу"; рассылка (дашборд)):
		172861 (172811)
	Субъекты, не отображаемые на дашборде:
                      Название субъекта РФ  session_id
Федеральное медико-биологическое агентство          50

Файл И38_успешные_сессии_20240213.csv:
		всего строк:		162161
		всего столбцов:		9 (верно)
	Пропуски данных (всего (доля от общего числа строк)):
		Дата:			0	(0.0 %)
		session_id:		0	(0.0 %)
		Название субъекта РФ:	0	(0.0 %)
		sp_oid:			2373	(1.46 %)
		sp_name:		0	(0.0 %)
		post_name:		0	(0.0 %)
		slot_ts:		0	(0.0 %)
		create_ts:		0	(0.0 %)
		Тип записи:		0	(0.0 %)
	Количество успешных 

#### Сравнение успехов "вчера-сегодня"

In [7]:
df_s_y = pd.pivot_table(
    df_s_yesterday,
    index="Название субъекта РФ",
    values="session_id",
    aggfunc="count",
).reset_index()
df_s_t = pd.pivot_table(
    df_s_today,
    index="Название субъекта РФ",
    values="session_id",
    aggfunc="count",
).reset_index()
df_s_yt = df_s_t.merge(df_s_y, on="Название субъекта РФ", how="outer").fillna(0)
df_s_yt = df_s_yt.rename(columns={"session_id_x": "усп отч", "session_id_y": "усп прш"})
df_s_yt["Δусп"] = df_s_yt["усп отч"] - df_s_yt["усп прш"]
df_s_yt

,Название субъекта РФ,усп отч,усп прш,Δусп
0,Алтайский край,6804,2866.0,3938.0
1,Амурская область,1030,495.0,535.0
2,Архангельская область,2259,603.0,1656.0
3,Астраханская область,61,25.0,36.0
4,Белгородская область,173,91.0,82.0
...,...,...,...,...
81,Ямало-Ненецкий автономный округ,277,38.0,239.0
82,Ярославская область,1741,861.0,880.0
83,г. Москва,2557,1551.0,1006.0
84,г. Санкт-Петербург,3389,1410.0,1979.0


### Сравнение результатов записи к врачу "вчера-сегодня" (без витрин)

In [8]:
df_yt = df_s_yt.merge(df_e_yt, on="Название субъекта РФ", how="outer")
df_yt

,Название субъекта РФ,усп отч,усп прш,Δусп,бп отч,бп прш,Δбп,оДБ отч,оДБ прш,ΔоДБ,...,ΔчДБ,т отч,т прш,Δт,тДБ отч,тДБ прш,ΔтДБ,фДБ отч,фДБ прш,ΔфДБ
0,Алтайский край,6804,2866.0,3938.0,5,1,4,387,97,290,...,19302,522,81,441,522,81,441,1128,388,740
1,Амурская область,1030,495.0,535.0,126,61,65,18,5,13,...,2021,463,651,-188,461,651,-190,56,15,41
2,Архангельская область,2259,603.0,1656.0,63,27,36,582,305,277,...,10390,160,59,101,119,56,63,857,235,622
3,Астраханская область,61,25.0,36.0,9,2,7,85,69,16,...,181,5,7,-2,5,7,-2,26,2,24
4,Белгородская область,173,91.0,82.0,16,14,2,52,17,35,...,293,468,44,424,468,44,424,1027,459,568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,Ямало-Ненецкий автономный округ,277,38.0,239.0,57,7,50,381,195,186,...,1431,17,5,12,16,5,11,452,76,376
82,Ярославская область,1741,861.0,880.0,78,48,30,2305,861,1444,...,1975,201,169,32,199,167,32,604,300,304
83,г. Москва,2557,1551.0,1006.0,0,0,0,1109,784,325,...,4252,5749,3275,2474,5749,3275,2474,275,87,188
84,г. Санкт-Петербург,3389,1410.0,1979.0,285,109,176,2084,1002,1082,...,9139,2395,514,1881,2369,510,1859,2540,847,1693


### Витрины

In [8]:
df_dm = pd.read_excel(
    dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0],
    sheet_name="Сессии по датам",
    index_col=None,
)
print("Файл: ...", max(os.listdir(dm_path)) + "/" + dm_file_last[0])
print(f"\t\tвсего строк:\t\t{df_dm.shape[0]}")
cnt_dm = "верно" if df_dm.shape[1] == 12 else "ошибка"
print(f"\t\tвсего столбцов:\t\t{df_dm.shape[1]} ({cnt_dm})")
print(
    f'\t\tдиапазон дат: \t\t{str(df_dm["Дата"].min())[:10]} - {str(df_dm["Дата"].max())[:10]}'
)
if "Шаг 4. Запись. Уникальные сессии" not in df_dm["Шаг"].unique():
    print("\t\tназвание шага 4:\tотличается от используемого для расчётов")
else:
    print("\t\tназвание шага 4:\t'Шаг 4. Запись. Уникальные сессии' - верное")
# названи шагов из файла
step_name_list = df_dm["Шаг"].unique()
print("\tНазвания шагов, используемые в файле:")
for step_name in step_name_list:
    print("\t\t", step_name)
# пропуски данных
print(
    f'\tПропуски данных за весь период и за {str(df_dm["Дата"].max())[:10]} (всего (доля от общего числа строк)):'
)
missing_dm_dict = {}
missing_dm_lastdate_dict = {}
df_dm_lastdate = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]
for column in df_dm.columns:
    missing_dm_dict[column] = [
        df_dm[column].isnull().sum(),
        round(df_dm[column].isnull().sum() * 100 / df_dm.shape[0], 2),
    ]
    missing_dm_lastdate_dict[column] = [
        df_dm_lastdate[column].isnull().sum(),
        round(df_dm_lastdate[column].isnull().sum() * 100 / df_dm_lastdate.shape[0], 2),
    ]
print(f'\t\t\t\t\tвесь период\t\t{str(df_dm["Дата"].max())[:10]}')

print(
    f'\t\tКод региона:\t\t{missing_dm_dict["Код региона"][0]}\t({missing_dm_dict["Код региона"][1]} %)\t\t{missing_dm_lastdate_dict["Код региона"][0]}\t({missing_dm_lastdate_dict["Код региона"][1]} %)'
)
print(
    f'\t\tРегион:\t\t\t{missing_dm_dict["Регион"][0]}\t({missing_dm_dict["Регион"][1]} %)\t\t{missing_dm_lastdate_dict["Регион"][0]}\t({missing_dm_lastdate_dict["Регион"][1]} %)'
)
print(
    f'\t\tДата:\t\t\t{missing_dm_dict["Дата"][0]}\t({missing_dm_dict["Дата"][1]} %)\t\t{missing_dm_lastdate_dict["Дата"][0]}\t({missing_dm_lastdate_dict["Дата"][1]} %)'
)
print(
    f'\t\tШаг:\t\t\t{missing_dm_dict["Шаг"][0]}\t({missing_dm_dict["Шаг"][1]} %)\t\t{missing_dm_lastdate_dict["Шаг"][0]}\t({missing_dm_lastdate_dict["Шаг"][1]} %)'
)
print(
    f'\t\tВсего:\t\t\t{missing_dm_dict["Всего"][0]}\t({missing_dm_dict["Всего"][1]} %)\t\t{missing_dm_lastdate_dict["Всего"][0]}\t({missing_dm_lastdate_dict["Всего"][1]} %)'
)
print(
    f'\t\tУспешно:\t\t{missing_dm_dict["Успешно"][0]}\t({missing_dm_dict["Успешно"][1]} %)\t\t{missing_dm_lastdate_dict["Успешно"][0]}\t({missing_dm_lastdate_dict["Успешно"][1]} %)'
)
print(
    f'\t\tОшибки ЕПГУ:\t\t{missing_dm_dict["Ошибки ЕПГУ"][0]}\t({missing_dm_dict["Ошибки ЕПГУ"][1]} %)\t\t{missing_dm_lastdate_dict["Ошибки ЕПГУ"][0]}\t({missing_dm_lastdate_dict["Ошибки ЕПГУ"][1]} %)'
)
print(
    f'\t\tТех ошибки:\t\t{missing_dm_dict["Тех ошибки"][0]}\t({missing_dm_dict["Тех ошибки"][1]} %)\t\t{missing_dm_lastdate_dict["Тех ошибки"][0]}\t({missing_dm_lastdate_dict["Тех ошибки"][1]} %)'
)
print(
    f'\t\tОрг ошибки:\t\t{missing_dm_dict["Орг ошибки"][0]}\t({missing_dm_dict["Орг ошибки"][1]} %)\t\t{missing_dm_lastdate_dict["Орг ошибки"][0]}\t({missing_dm_lastdate_dict["Орг ошибки"][1]} %)'
)
print(
    f'\t\tПОДД:\t\t\t{missing_dm_dict["ПОДД"][0]}\t({missing_dm_dict["ПОДД"][1]} %)\t\t{missing_dm_lastdate_dict["ПОДД"][0]}\t({missing_dm_lastdate_dict["ПОДД"][1]} %)'
)
print(
    f'\t\tБрошенные:\t\t{missing_dm_dict["Брошенные"][0]}\t({missing_dm_dict["Брошенные"][1]} %)\t\t{missing_dm_lastdate_dict["Брошенные"][0]}\t({missing_dm_lastdate_dict["Брошенные"][1]} %)'
)
print(
    f'\t\tБизнес-правила:\t\t{missing_dm_dict["Бизнес-правила"][0]}\t({missing_dm_dict["Бизнес-правила"][1]} %)\t{missing_dm_lastdate_dict["Бизнес-правила"][0]}\t({missing_dm_lastdate_dict["Бизнес-правила"][1]} %)'
)
print(
    f'\tРезультаты записи на приём с использованием витрин ЕПГУ на {str(df_dm["Дата"].max())[:10]}'
)
print("\t(бизнес-правила игнорируются):")
# количество успешных записей
dm_s = df_dm[
    (df_dm["Дата"] == df_dm["Дата"].max()) & (df_dm["Шаг"] == step_name_list[3])
]["Успешно"].sum()
# количество технических ошибок
dm_e_t = (
    df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Тех ошибки"].sum()
    + df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["ПОДД"].sum()
)
# количество организационных ошибок
dm_e_o = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Орг ошибки"].sum()
# количество ошибок ФЭР
dm_e_r = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Ошибки ЕПГУ"].sum()
# черновики
dm_e_d = df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Брошенные"].sum()
# бизнес-правила
dm_e_b = int(df_dm[df_dm["Дата"] == df_dm["Дата"].max()]["Бизнес-правила"].sum())
print(f"\t\tуспешные записи:\t{dm_s}")
print(f"\t\tтехнические ошибки:\t{dm_e_t}")
print(f"\t\tорганизационные ошибки:\t{dm_e_o}")
print(f"\t\tошибки ФЭР:\t\t{dm_e_r}")
print(f"\t\tчерновики:\t\t{dm_e_d}")
print(f"\t\tбизнес-правила:\t\t{dm_e_b}")

# проверка названий регионов
df_dm_regname_ref = pd.read_csv(vis_reg_ref_path, sep=";")
print(
    "Есть в актуальном файле витрин, но нет в чат-боте (не будут распознаны и рассчитаны):"
)
df_dm_regname_actual = pd.DataFrame(df_dm["Регион"].unique().tolist()).rename(
    columns={0: "dm_reg_name_act"}
)
df_dm_regname_contrast = df_dm_regname_actual.merge(
    df_dm_regname_ref, left_on="dm_reg_name_act", right_on="dm_reg_name_ref", how="left"
)
df_dm_regname_contrast_print = df_dm_regname_contrast[
    df_dm_regname_contrast["dm_reg_name_ref"].isnull()
]
if df_dm_regname_contrast_print.empty:
    print(
        f"в файле данных витрин нет регионов, не указанных в справочнике {vis_reg_ref_path}"
    )
else:
    print(df_dm_regname_contrast_print.to_string(index=False))

badsim(df_dm, "_")

Файл: ... 2024-02-13/Ежедневные_данные_по_сессиям.xlsx
		всего строк:		28628
		всего столбцов:		12 (верно)
		диапазон дат: 		2023-06-12 - 2024-02-13
		название шага 4:	'Шаг 4. Запись. Уникальные сессии' - верное
	Названия шагов, используемые в файле:
		 Шаг 1. Идентификация пациента в регионе. Уникальные сессии
		 Шаг 2. Выбор специальности. Уникальные сессии
		 Шаг 3. Выбор врача и времени. Уникальные сессии
		 Шаг 4. Запись. Уникальные сессии
	Пропуски данных за весь период и за 2024-02-13 (всего (доля от общего числа строк)):
					весь период		2024-02-13
		Код региона:		0	(0.0 %)		0	(0.0 %)
		Регион:			0	(0.0 %)		0	(0.0 %)
		Дата:			0	(0.0 %)		0	(0.0 %)
		Шаг:			0	(0.0 %)		0	(0.0 %)
		Всего:			0	(0.0 %)		0	(0.0 %)
		Успешно:		0	(0.0 %)		0	(0.0 %)
		Ошибки ЕПГУ:		0	(0.0 %)		0	(0.0 %)
		Тех ошибки:		0	(0.0 %)		0	(0.0 %)
		Орг ошибки:		0	(0.0 %)		0	(0.0 %)
		ПОДД:			0	(0.0 %)		0	(0.0 %)
		Брошенные:		0	(0.0 %)		0	(0.0 %)
		Бизнес-правила:		18308	(63.95 %)	0	(0.0 %)
	Результаты записи н

#### Автоматизированная загрузка списка витринных регионов из последнего файла чат-бота

In [9]:
viz_dir_path = "C:/work/new_bot/new"
viz_file_list = os.listdir(viz_dir_path)
macros_files_dict = {}
for filename in viz_file_list:
    if filename.split(".")[-1] == "xlsm":
        macros_files_dict[os.path.getmtime(viz_dir_path + "/" + filename)] = filename
last_macros_path = viz_dir_path + "/" + macros_files_dict[max(macros_files_dict.keys())]
try:
    df_regname_macros = pd.read_excel(
        last_macros_path,
        sheet_name="СС",
        index_col=None,
    )
except Exception as e:
    sys.exit(e)
print(f"используется файл чат-бота:\t{last_macros_path}")
# список регионов с листа СС макроса
df_dm_regname_macros = (
    pd.DataFrame(df_regname_macros["Unnamed: 4"])
    .dropna()
    .rename(columns={"Unnamed: 4": "macros_reg_name"})
)
# список регионов из моего справочника
df_dm_regname_ref = pd.read_csv(vis_reg_ref_path, sep=";")
print(f"используется справочник:\t{vis_reg_ref_path}")

# список регионов из актуального исходника витрин
df_dm = pd.read_excel(
    dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0],
    sheet_name="Сессии по датам",
    index_col=None,
)
df_dm_regname_actual = pd.DataFrame(df_dm["Регион"].unique().tolist()).rename(
    columns={0: "dm_reg_name_act"}
)
dm_path_used = dm_path + "/" + max(os.listdir(dm_path)) + "/" + dm_file_last[0]
print(f"используется исходник витрин:\t{dm_path_used}")
df_dm_full_reg = df_dm_regname_actual.merge(
    df_dm_regname_macros,
    left_on="dm_reg_name_act",
    right_on="macros_reg_name",
    how="outer",
).merge(
    df_dm_regname_ref,
    left_on="dm_reg_name_act",
    right_on="dm_reg_name_ref",
    how="outer",
)
df_dm_full_reg.sort_values("dm_reg_name_act", inplace=True)
df_dm_full_reg.rename(
    columns={
        "dm_reg_name_act": "есть в исходнике витрин",
        "macros_reg_name": "есть в файле чат-бота",
        "dm_reg_name_ref": "есть в моём справочнике",
    },
    inplace=True,
)
# print(df_dm_full_reg.to_string(index=False))
df_dm_full_reg.reset_index(drop=True)

используется файл чат-бота:	C:/work/new_bot/new/ЭЗ_бот_2024-02-13.xlsm
используется справочник:	C:/work/new_bot/new/vis_reg_ref.csv
используется исходник витрин:	C:/work/new_bot/new/dm/2024-02-13/Ежедневные_данные_по_сессиям.xlsx


,есть в исходнике витрин,есть в файле чат-бота,есть в моём справочнике
0,Астраханская область,Астраханская область,Астраханская область
1,Белгородская область,Белгородская область,Белгородская область
2,Брянская область,Брянская область,Брянская область
3,Владимирская область,Владимирская область,Владимирская область
4,Волгоградская область,Волгоградская область,Волгоградская область
5,Вологодская область,Вологодская область,Вологодская область
6,Воронежская область,Воронежская область,Воронежская область
7,Еврейская автономная область,Еврейская автономная область,Еврейская автономная область
8,Иркутская область,Иркутская область,Иркутская область
9,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика,Кабардино-Балкарская Республика


### Выгрузки ПОС

In [13]:
df_pos_ref_reg = pd.read_csv(pos_reg_ref_path, sep=";")
df_pos = pd.DataFrame()
for file in pos_dir_last:
    df_pos_tmp = pd.read_excel(
        pos_path + "/" + max(os.listdir(pos_path)) + "/" + file,
        sheet_name="Sheet0",
        index_col=None,
    )
    df_pos = pd.concat([df_pos, df_pos_tmp])
    print(f"Загружен файл {file}")
df_pos.reset_index(drop=True, inplace=True)
df_pos_act_reg = pd.DataFrame(df_pos["Верхнеуровневый ЛКО"].unique()).rename(
    columns={0: "actual_reg_name"}
)
df_reg_pos_contrast = df_pos_act_reg.merge(
    df_pos_ref_reg, left_on="actual_reg_name", right_on="pos_reg_name", how="left"
)
no_regnames = df_reg_pos_contrast[df_reg_pos_contrast["pos_reg_name"].isnull()][
    "actual_reg_name"
].tolist()
print(f"Отсутствуют в актуальном списке регионов для расчёта рассылки ({pos_reg_ref_path}):")
for name in no_regnames:
    print("\t", name)

# проверка на "_"
badsim(df_pos, "_")

Загружен файл Выгрузка ПОС 01.03.2023 по 31.05.2023.xlsx
Загружен файл Выгрузка ПОС 01.06.2023 по 31.08.2023.xlsx
Загружен файл Выгрузка ПОС 01.09.2023 по 30.11.2023.xlsx
Загружен файл Выгрузка ПОС 01.12.2023 по 12.02.2024.xlsx
Загружен файл Выгрузка ПОС 11.11.2022 по 28.02.2023.xlsx
Отсутствуют в актуальном списке регионов для расчёта рассылки (C:/work/pos_bot/pos_reg_ref.csv):
	 ФЕДЕРАЛЬНОЕ МЕДИКО-БИОЛОГИЧЕСКОЕ АГЕНТСТВО
	 АДМИНИСТРАЦИЯ ГУБЕРНАТОРА ТЕСТОВОГО РЕГИОНА
	 АППАРАТ ПРАВИТЕЛЬСТВА ДОНЕЦКОЙ НАРОДНОЙ РЕСПУБЛИКИ
	 ВОЕННО-ГРАЖДАНСКАЯ АДМИНИСТРАЦИЯ ХЕРСОНСКОЙ ОБЛАСТИ
	 АДМИНИСТРАЦИЯ ГЛАВЫ ЛУГАНСКОЙ НАРОДНОЙ РЕСПУБЛИКИ
Записи, содержащие '_':
	ПОЛЕ: Источник
['KAMCHATSKIY_KRAY' 'MED_DOC_EPGU']
	ПОЛЕ: Верхнеуровневый ЛКО
['Аппарат Губернатора и Правительства Оренбургской области_Аппарат Губернатора и Правительства Оренбургской области']
	ПОЛЕ: Организация, в которую поступило сообщение
['МИНЗДРАВ ОРЕНБУРГСКОЙ ОБЛАСТИ_МИНЗДРАВ ОРЕНБУРГСКОЙ ОБЛАСТИ'
 'МИНЗДРАВ ОРЕНБУРГСКОЙ ОБЛАСТИ_ГО

***

### **Ситуативная сверка с дашбордом**

In [25]:
# загрузка данных из БД38
df_db38 = pd.read_excel(
    "C:/work/comparison_error_successful/db_data/db_data.xlsx",
    sheet_name="1",
    index_col=None,
).fillna(0)
df_db38["b"] = df_db38["b"].astype("int")
df_db38.insert(
    2, "end_date", df_db38["begin_date"].apply(lambda x: x + timedelta(days=6))
)
df_db38 = df_db38.drop("per", axis=1)
df_db38["date_interval"] = df_db38.apply(
    lambda x: x["begin_date"].strftime("%Y-%m-%d")
    + "_"
    + x["end_date"].strftime("%Y-%m-%d"),
    axis=1,
)
df_db38_group = (
    df_db38[["date_interval", "s", "d", "o", "t", "b", "r"]]
    .groupby("date_interval")
    .sum()
    .reset_index()
)
df_db38_group

,date_interval,s,d,o,t,b,r
0,2022-08-22_2022-08-28,392934,1490942,522569,476508,0,98630
1,2022-08-29_2022-09-04,406878,1477808,495559,453596,0,60952
2,2022-09-05_2022-09-11,440400,1654386,531767,501942,0,110253
3,2022-09-12_2022-09-18,448675,1696666,530951,618780,0,107420
4,2022-09-19_2022-09-25,446461,1615018,509438,547847,0,115686
...,...,...,...,...,...,...,...
70,2023-12-25_2023-12-31,381976,1308565,139095,179895,29540,80668
71,2024-01-01_2024-01-07,396916,950536,87188,113438,21488,55825
72,2024-01-08_2024-01-14,917724,2188512,256120,297006,65811,143633
73,2024-01-15_2024-01-21,872500,2282921,241923,292696,64892,156722


In [26]:
# то же по регионам
df_db38_group_region = (
    df_db38[["region", "s", "d", "o", "t", "b", "r", "date_interval"]]
    .groupby(["region", "date_interval"])
    .sum()
    .reset_index()
)
df_db38_group_region

,region,date_interval,s,d,o,t,b,r
0,Алтайский край,2022-08-22_2022-08-28,15602,100274,5641,4456,0,2645
1,Алтайский край,2022-08-29_2022-09-04,15180,92615,5731,4043,0,2190
2,Алтайский край,2022-09-05_2022-09-11,16444,100234,6523,5214,0,6608
3,Алтайский край,2022-09-12_2022-09-18,15700,105435,6513,20883,0,4115
4,Алтайский край,2022-09-19_2022-09-25,16358,93605,5539,3499,0,7946
...,...,...,...,...,...,...,...,...
6370,г. Севастополь,2023-12-25_2023-12-31,3640,18329,1236,565,187,1103
6371,г. Севастополь,2024-01-01_2024-01-07,6013,14256,223,272,269,720
6372,г. Севастополь,2024-01-08_2024-01-14,9398,25002,471,573,769,1316
6373,г. Севастополь,2024-01-15_2024-01-21,9313,30516,643,447,688,1727


In [27]:
# загрузка из выгрузки дашборда
df_dash = pd.read_excel(
    "C:/work/comparison_error_successful/db_data/dash_data.xlsx",
    sheet_name="1",
    index_col=None,
)
df_dash["date"] = pd.to_datetime(df_dash["date"], format="%Y-%m-%d")
df_dash = df_dash.sort_values("date")

# добавление столбца с интервалами по 7 дней
df_dash_int = pd.DataFrame()
for i in range(0, df_dash.shape[0] // 7 * 7, 7):
    index_list = [x for x in range(i, i + 7)]
    df_tmp = df_dash.iloc[index_list]
    date_begin = df_tmp["date"].min()
    date_end = df_tmp["date"].max()
    date_interval = (
        date_begin.strftime("%Y-%m-%d") + "_" + date_end.strftime("%Y-%m-%d")
    )
    df_tmp["date_interval"] = date_interval
    df_dash_int = pd.concat([df_dash_int, df_tmp])

df_dash_group = (
    df_dash_int[["date_interval", "o_d", "r_d", "t_d", "s_d"]]
    .groupby("date_interval")
    .sum()
    .reset_index()
)
df_dash_group

,date_interval,o_d,r_d,t_d,s_d
0,2022-08-22_2022-08-28,522569,98922,478559,396534
1,2022-08-29_2022-09-04,495559,61014,456852,406878
2,2022-09-05_2022-09-11,531768,110396,505355,440400
3,2022-09-12_2022-09-18,531423,107755,624034,448675
4,2022-09-19_2022-09-25,510189,115942,555194,446461
...,...,...,...,...,...
70,2023-12-25_2023-12-31,139095,80671,173974,381976
71,2024-01-01_2024-01-07,87188,55828,109347,396916
72,2024-01-08_2024-01-14,256120,143636,287987,917724
73,2024-01-15_2024-01-21,241923,156727,282077,872500


In [30]:
# объединение и расчёт расхождений
df_test = df_dash_group.merge(
    df_db38_group[["date_interval", "o", "r", "t", "s"]],
    on="date_interval",
    how="outer",
)
df_test = df_test[["date_interval", "s_d", "s", "o_d", "o", "t_d", "t", "r_d", "r"]]
df_test.insert(3, "Δs", df_test["s_d"] - df_test["s"])
df_test.insert(6, "Δo", df_test["o_d"] - df_test["o"])
df_test.insert(9, "Δt", df_test["t_d"] - df_test["t"])
df_test["Δr"] = df_test["r_d"] - df_test["r"]
df_test

,date_interval,s_d,s,Δs,o_d,o,Δo,t_d,t,Δt,r_d,r,Δr
0,2022-08-22_2022-08-28,396534,392934,3600,522569,522569,0,478559,476508,2051,98922,98630,292
1,2022-08-29_2022-09-04,406878,406878,0,495559,495559,0,456852,453596,3256,61014,60952,62
2,2022-09-05_2022-09-11,440400,440400,0,531768,531767,1,505355,501942,3413,110396,110253,143
3,2022-09-12_2022-09-18,448675,448675,0,531423,530951,472,624034,618780,5254,107755,107420,335
4,2022-09-19_2022-09-25,446461,446461,0,510189,509438,751,555194,547847,7347,115942,115686,256
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,2023-12-25_2023-12-31,381976,381976,0,139095,139095,0,173974,179895,-5921,80671,80668,3
71,2024-01-01_2024-01-07,396916,396916,0,87188,87188,0,109347,113438,-4091,55828,55825,3
72,2024-01-08_2024-01-14,917724,917724,0,256120,256120,0,287987,297006,-9019,143636,143633,3
73,2024-01-15_2024-01-21,872500,872500,0,241923,241923,0,282077,292696,-10619,156727,156722,5


In [37]:
# загрузка моей предрасчитанной статистики с 6 ноября по 31 декабря
# без Байконура и ФМБА, но с Кемской волостью

stat_11_12_path = "C:/work/comparison_error_successful/data_11_12/data_11-12_weeks.csv"

df_stat_11_12 = pd.read_csv(stat_11_12_path, sep=";")
# без Кемской волости
df_stat_11_12 = df_stat_11_12[df_stat_11_12["real_region_name"] != "Кемская волость"]


df_stat_11_12["date"] = pd.to_datetime(df_stat_11_12["date"], format="%d.%m.%Y")


df_stat_11_12_p = (
    pd.pivot_table(
        df_stat_11_12, index="date", columns="type", values="value", aggfunc="sum"
    )
    .reset_index()
    .rename_axis(None, axis=1)
)

df_stat_11_12_int = pd.DataFrame()
for i in range(0, df_stat_11_12_p.shape[0] // 7 * 7, 7):
    index_list = [x for x in range(i, i + 7)]
    df_tmp = df_stat_11_12_p.iloc[index_list]
    date_begin = df_tmp["date"].min()
    date_end = df_tmp["date"].max()
    date_interval = (
        date_begin.strftime("%Y-%m-%d") + "_" + date_end.strftime("%Y-%m-%d")
    )
    df_tmp["date_interval"] = date_interval
    df_stat_11_12_int = pd.concat([df_stat_11_12_int, df_tmp])
df_stat_11_12_int = df_stat_11_12_int.rename(
    columns={
        "Организационные ошибки": "o_my",
        "Ошибки ФЭР": "r_my",
        "Технические ошибки": "t_my",
        "Успешные записи": "s_my",
        "Черновики": "d_my",
    }
)
df_stat_11_12_group = (
    df_stat_11_12_int[["date_interval", "o_my", "r_my", "t_my", "s_my", "d_my"]]
    .groupby("date_interval")
    .sum()
)
df_stat_11_12_group = df_stat_11_12_group[
    ["s_my", "o_my", "t_my", "r_my"]
].reset_index()
df_stat_11_12_group

,date_interval,s_my,o_my,t_my,r_my
0,2023-11-06_2023-11-12,779713,282955,325562,158005
1,2023-11-13_2023-11-19,806094,269930,333416,207319
2,2023-11-20_2023-11-26,838960,284453,453286,167827
3,2023-11-27_2023-12-03,878490,290400,430188,168240
4,2023-12-04_2023-12-10,830193,250524,462139,138220
5,2023-12-11_2023-12-17,816697,197996,409735,143082
6,2023-12-18_2023-12-24,592523,207482,250561,120517
7,2023-12-25_2023-12-31,381976,139095,179895,80668


In [35]:
df_stat_11_12[df_stat_11_12["real_region_name"] == "Кемская волость"][["type", "value"]].groupby("type").sum()

,value
type,
Организационные ошибки,0
Ошибки ФЭР,29
Технические ошибки,0
Успешные записи,0
Черновики,0
